## Hello World

In [1]:
import asyncio
import random

async def say(what, when):
    await asyncio.sleep(when)
    print(what)
    
loop = asyncio.get_event_loop()    
loop.run_until_complete(say("hello World", 1))

hello World


## Create tasks

- `ensure_future` is method to create `Task` from coroutine. It creates task different ways based on argument (including using of `create_task` for coroutines and future-like objects).

- `create_task` is abstract method of `AbstractEventLoop`. Different event loops can implement this function different ways.

- You should use `ensure_future` to create tasks. You'll need `create_task` only if you're going to implement your own event loop type.

In [3]:
async def main(loop):
    task1 = asyncio.ensure_future(say('first hello', 3))
    task2 = asyncio.ensure_future(say('second hello', 2))
    await task1, task2
    
loop.run_until_complete(main(loop))

second hello
first hello


In [4]:
async def stop(loop, when):
    await asyncio.sleep(when)
    print('loop.stop()')
    loop.stop()
    
loop.create_task(say('first hello', 2))
loop.create_task(say('second hello', 1))
loop.create_task(stop(loop, 5))
loop.run_forever()

second hello
first hello
loop.stop()


## asyncio Producer Consumer model

In [5]:
async def produce(queue, n, num):
    for x in range(1, n + 1):
        print('producing {}/{}:{}'.format(x, n, num))
        await asyncio.sleep(random.random())
        await queue.put(str(x))

    await queue.put(None)


async def consume(queue, num):
    while True:
        item = await queue.get()
        if item is None:
            break
        print('consuming item {}/{}...'.format(item, num))
        await asyncio.sleep(random.random())


loop = asyncio.get_event_loop()
queue = asyncio.Queue(loop=loop)
producer_coro1 = produce(queue, 4, 1)
producer_coro2 = produce(queue, 4, 2)
consumer_coro1 = consume(queue, 1)
consumer_coro2 = consume(queue, 2)
loop.run_until_complete(asyncio.gather(producer_coro1, consumer_coro1, producer_coro2, consumer_coro2))

producing 1/4:1
producing 1/4:2
producing 2/4:1
producing 2/4:2
consuming item 1/1...
consuming item 1/2...
producing 3/4:2
producing 3/4:1
producing 4/4:2
consuming item 2/1...
consuming item 2/2...
consuming item 3/1...
consuming item 4/2...
producing 4/4:1
consuming item 3/1...
consuming item 4/1...


[None, None, None, None]